# Preprocessing and models

In [0]:
%tensorflow_version 1.x
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

import json
import os
import pickle

import cv2
import numpy as np
from keras import metrics, Model
from keras.applications.vgg16 import preprocess_input, VGG16
from keras.callbacks import EarlyStopping
from keras.datasets import fashion_mnist
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import load_model, Sequential
from keras.preprocessing import image as image_manip
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


OUTPUT_DIR = 'drive/My Drive/fashion-mnist-classifier'
ACTIVATION_FUNCTION = 'relu'
BATCH_SIZE = 64
EPOCHS = 100
EARLY_STOPPING = EarlyStopping(patience=10, min_delta=0.001)


def prepare_gabor_filters():
    filters = []
    thetas = [np.pi * i / 8 for i in range(8)]
    lambdas = np.arange(6.0, 11.0)
    sigmas = np.arange(1.0, 6.0)
    for theta in thetas:
        for lambd in lambdas:
            for sigma in sigmas:
                filters.append(cv2.getGaborKernel((9, 9), sigma, theta, lambd, 0.5, 0, ktype=cv2.CV_32F))
    return filters


def extract_gabor_features(dataset):
    features = []
    for image in dataset:
        features_vector = []
        height = len(image)
        width = len(image[0])
        number_of_pixels = height * width
        for gabor_filter in gabor_filters:
            filtered_image = cv2.filter2D(image, cv2.CV_8UC1, gabor_filter)
            # mean energy
            features_vector.append(np.mean(filtered_image.astype('float64')**2) / number_of_pixels / number_of_pixels)
            # mean_amplitude
            features_vector.append(np.mean(filtered_image) / number_of_pixels)
        features.append(features_vector)
    return np.array(features)


def rescale_and_preprocess_images(dataset, resize_dim):
    """Resize image to size required by VGG16 net and apply the same preprocessing as in original net"""
    preproceses_images = []
    for image in dataset:
        image_preprocessed = cv2.resize(image, (resize_dim, resize_dim))
        image_preprocessed = image_manip.img_to_array(image_preprocessed)
        image_preprocessed = np.expand_dims(image_preprocessed, axis=0)
        image_preprocessed = preprocess_input(image_preprocessed)
        preproceses_images.append(image_preprocessed[0])
    return np.array(preproceses_images)


def load_data(model_type, save_data=False):

    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

    if model_type == 'mlp':
        
        gabor_filters = prepare_gabor_filters()

        x_train = extract_gabor_features(x_train)
        x_test = extract_gabor_features(x_test)

        suffix = ''

    elif model_type == 'vgg_16':

        # rescale to three channels
        x_train = np.stack((x_train,)*3, axis=-1)
        x_test = np.stack((x_test,)*3, axis=-1)

        # vgg16 requires images to be at least 32x32 (max pooling 5 times) -> reshaping images to 56x56 should suffice
        x_train = rescale_and_preprocess_images(x_train, x_train.shape[1] * 2)
        x_test = rescale_and_preprocess_images(x_test, x_test.shape[1] * 2)

        suffix = '_vgg_16'

    else:
        raise ValueError('Unknown model type')

    number_of_classes = len(np.unique(np.append(y_train, y_test)))

    y_train = np_utils.to_categorical(y_train, number_of_classes)
    y_test = np_utils.to_categorical(y_test, number_of_classes)

    if save_data:
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)
        np.save(os.path.join(OUTPUT_DIR, 'x_train{}'.format(suffix)), x_train)
        np.save(os.path.join(OUTPUT_DIR, 'y_train{}'.format(suffix)), y_train)
        np.save(os.path.join(OUTPUT_DIR, 'x_test{}'.format(suffix)), x_test)
        np.save(os.path.join(OUTPUT_DIR, 'y_test{}'.format(suffix)), y_test)

    return x_train, y_train, x_test, y_test


def top_1_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=1)


def top_5_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)

   
def build_mlp_model(input_shape, layer_sizes, activation_function, dropout, number_of_classes):
    model = Sequential()
    model.add(Dense(layer_sizes[0], activation=activation_function, input_shape=(input_shape)))
    # smaller dropout on input layer
    model.add(Dropout(min(dropout, 0.2)))
    for layer_size in layer_sizes[1:]:
        model.add(Dense(layer_size, activation=activation_function))
        model.add(Dropout(dropout))
    model.add(Dense(number_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[top_1_accuracy, top_5_accuracy])
    return model
    

def build_vgg16_model(input_shape, layer_sizes, activation_function, dropout, number_of_classes, train_convolutions=False):

    vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
   
    # set convolutional layers as nontrainable
    if not train_convolutions:
        for layer in vgg16_model.layers:
            layer.trainable = False

    x = vgg16_model.layers[-1].output

    # descriptor and classifier
    x = GlobalAveragePooling2D()(x)

    x = Dense(layer_sizes[0], activation=activation_function)(x)
    x = Dropout(min(dropout, 0.2))(x)
    for layer_size in layer_sizes[1:]:
        x = Dense(layer_size, activation=activation_function)(x)
        x = Dropout(dropout)(x)

    predictions = Dense(number_of_classes, activation='softmax')(x)

    model = Model(inputs=vgg16_model.input, outputs=predictions)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[top_1_accuracy, top_5_accuracy])
    print('Model loaded')
    
    return model


def execute_experiment(model_type, x_train, y_train, layer_counts, layer_sizes, dropouts=[0.0], save_models=False, test_size=0.2):

    for collection in [layer_counts, layer_sizes, dropouts]:
        if not isinstance(collection, list) or not collection:
            raise TypeError('Non empty list is expected for layer_counts, layer_sizes and dropouts')
    
    if any([dropout for dropout in dropouts 
            if dropout < 0.0 or dropout > 1.0]):
        raise ValueError('dropouts must be between 0.0 and 1.0')

    if any([layer_size for layer_size in layer_sizes 
            if layer_size <= 0 or not isinstance(layer_size, int)]):
        raise ValueError('layer_sizes must be positive integers')

    if any([number_of_layers for number_of_layers in layer_counts 
            if number_of_layers <= 0 or not isinstance(number_of_layers, int)]):
        raise ValueError('layer_counts must be positive integers')

    if model_type == 'mlp':

        input_shape = (x_train.shape[1],)
        model_function = build_mlp_model
        suffix = ''

    elif model_type == 'vgg_16':

        input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
        model_function = build_vgg16_model
        suffix = '_vgg_16'

    else:
        raise ValueError('Unknown model type')

    if save_models and not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    number_of_classes = y_train.shape[1]
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=test_size)

    scores = []

    for number_of_layers in layer_counts:
        for layer_size in layer_sizes:
            for dropout in dropouts:

                # adjust layer size to droput
                layers = [int(layer_size / (1.0 - dropout))] * number_of_layers
            
                model = model_function(input_shape, 
                                       layers, 
                                       ACTIVATION_FUNCTION,
                                       dropout, 
                                       number_of_classes)
               
                model.summary()

                history = model.fit(x_train,
                                    y_train,
                                    batch_size=BATCH_SIZE,
                                    epochs=EPOCHS,
                                    validation_data=(x_val, y_val),
                                    shuffle=True,
                                    verbose=1,
                                    callbacks=[EARLY_STOPPING])
 
                score = model.evaluate(x_test, y_test)

                scores.append({
                    'dropout': dropout,
                    'layer_size': layer_size,
                    'number_of_layers': number_of_layers,
                    'score': score
                })

                if save_models:
                    model.save(os.path.join(OUTPUT_DIR, 'model{}_{}_{}_{}.h5'.format(suffix, dropout, layer_size, number_of_layers)))

                    with open(os.path.join(OUTPUT_DIR, 'model{}_{}_{}_{}_history_dict.pkl'.format(suffix, dropout, layer_size, number_of_layers)), 'wb') as f:
                        pickle.dump(history.history, f)

    # return last model and scores
    return model, scores


def analyze_results(model, x_test, y_test):

    prob = model.predict(x_test, verbose=1) 
    y_pred = prob.argmax(axis=-1)

    y_test_labels = [np.where(test_case == 1)[0][0] for test_case in y_test]

    cm = confusion_matrix(y_test_labels, y_pred)

    accuracy = cm.diagonal().sum()/cm.sum()

    # accuracy per class
    class_accuracies = (cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]).diagonal()

    # classes sorted by their accuracy
    class_ranking = np.argsort(class_accuracies)

    class_accuracy_dict = [{'class': int(label), 'accuracy': float(class_accuracies[label])} for label in class_ranking]

    # most commonly made mistakes
    most_common_mistakes = [{'mistakes_count': int(cm[el//10][el%10]), 'expected_class': int(el//10), 'predicted_class': int(el%10)} for el in np.argsort(-cm, axis=None) if el//10 != el%10]

    return accuracy, cm, class_accuracy_dict, most_common_mistakes

Using TensorFlow backend.


# Data loading and feature extraction for MLPClassifier

In [0]:
x_train, y_train, x_test, y_test = load_data('mlp')

# First experiment: testing different layer number and sizes

In [0]:
layer_counts = [1, 2, 3, 4]

layer_sizes = [128, 256, 512, 1024]

model, scores = execute_experiment('mlp', x_train, y_train, layer_counts, layer_sizes)




Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128)               51328     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 52,618
Trainable params: 52,618
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 48000 samples, validate on 12000 samples
Epoch 1/100





48000/48000 [==============================] - 12s 247us/step - loss: 1.5539 - top_1_accuracy: 0.4262 - top_5_accuracy: 0.9237 - val_loss: 1.2219 - val_top_1_accuracy: 0.5408 - val_top_5_accuracy: 0.9680
Epoch 2/10

# Second experiment: VGG16 convolutional network

# Prepare data (for training or testing)

In [0]:
x_train, y_train, x_test, y_test = load_data('vgg_16')

4423680/4422102 [==============================] - 1s 0us/step


# Prepare and train model

In [0]:
# layer_sizes = [128, 256, 512, 1024]

# dropouts = [0, 0.1, 0.2, 0.3, 0.4, 0.5]

# best model achieved with drop probability equal to 0.2 and two layers 128 nodes
layer_counts = [2]

layer_sizes = [128]

dropouts = [0.2]

model, scores = execute_experiment('vgg_16', x_train, y_train, layer_counts, layer_sizes, dropouts)





58892288/58889256 [==============================] - 1s 0us/step







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model loaded
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 56, 56, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 56, 56, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 56, 56, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 28, 28, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 28, 28, 128)       73856     
___________________________________________________

# Analyze the results

In [0]:
accuracy, cm, class_accuracy_dict, most_common_mistakes = analyze_results(model, x_test, y_test)

print('accuracy: {}'.format(accuracy))
print('confuson_matrix:\n {}'.format(cm))
print('class_accuracy_dict:')
print(json.dumps(class_accuracy_dict, indent=4, sort_keys=True))
print('most_common_mistakes:')
print(json.dumps(most_common_mistakes[:10], indent=4, sort_keys=True))

10000/10000 [==============================] - 5s 457us/step
accuracy: 0.8691
confuson_matrix:
 [[843   1  19  36   5   0  85   0  11   0]
 [  5 964   1  19   4   0   2   0   5   0]
 [ 23   2 837  10  66   0  61   0   1   0]
 [ 39  10  25 819  63   0  39   0   4   1]
 [  6   0  94  28 773   0  99   0   0   0]
 [  1   0   0   0   0 939   0  49   0  11]
 [149   1  76  33  77   1 650   0  12   1]
 [  0   0   0   0   0  13   0 946   1  40]
 [  7   2   4   0   5   5   5   1 971   0]
 [  0   1   0   0   0  12   0  38   0 949]]
class_accuracy_dict:
[
    {
        "accuracy": 0.65,
        "class": 6
    },
    {
        "accuracy": 0.773,
        "class": 4
    },
    {
        "accuracy": 0.819,
        "class": 3
    },
    {
        "accuracy": 0.837,
        "class": 2
    },
    {
        "accuracy": 0.843,
        "class": 0
    },
    {
        "accuracy": 0.939,
        "class": 5
    },
    {
        "accuracy": 0.946,
        "class": 7
    },
    {
        "accuracy": 0.949,
     